# 신경망 모델 구성하기

신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성되어 있다. torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 모듈은 nn.Module의 하위 클래스(subclass)이다. 신경망은 다른 모듈로 수정된 모듈이다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있다.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### 학습을 위한 장치 얻기

가능한 경우 GPU 또는 MPS와 같은 하드웨어 가속기에서 모델을 학습하려고 한다. torch.cuda 또는 torch.backends.mas 가 사용 가능한지 확인해보고, 그렇지 않으면 CPU를 계속 사용한다.

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


### 클래스 정의하기

신경망 모델을 <code>nn.Module</code>의 하위 클래스로 정의하고, <code>____init____</code>에서 신경망 계층들을 초기화한다. <code>nn.Module</code>을 상속받은 모든 클래스는 <code>forward</code> 메소드에 입력 데이터에 대한 연산들을 구현한다.

In [3]:
class NeuralNetwork(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x) :
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

<code>NeuralNetwork</code>의 인스턴스(instance)를 생성하고 이를 <code>device</code>로 이동한 뒤, 구조(structure)를 출력한다.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델을 사용하기 위해 입력 데이터를 전달한다. 이는 일부 백그라운드 연산들과 함께 모델의 <code>forward</code>를 실행한다. <code>model.forward()</code>를 직접 호출하면 안된다!!

모델에 입력을 전달하여 호출하면 2차원 텐서를 반환한다. 2차원 텐서의 dim=0은 각 분류(class)에 대한 원시(raw) 예측값 10개가, dim=1에는 각 출력의 개별 값들이 해당한다. 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻는다.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class : {y_pred}")

Predicted class : tensor([2], device='cuda:0')


### 모델 계층(Layer)

FashionMNIST 모델의 계층들을 살펴보자. 이를 설명하기 위해 28X28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 보자.

In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

nn.Flatten 계층을 초기화하여 각 28X28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 반환한다. (dim=0의 미니배치 차원은 유지된다.)

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

선형 계층은 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈이다.

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만든다. 비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity)을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕는다.

이 모델에서는 nn.ReLU를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수도 있다.

In [9]:
print(f"Before ReLU : {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU : {hidden1}")

Before ReLU : tensor([[ 0.4024,  0.5842, -0.2439, -0.1010,  0.0062,  0.0806, -0.3972,  0.4268,
          0.2608, -0.2362,  0.2940, -0.0750,  0.1083, -0.2116,  0.5727,  0.1502,
          0.4755,  0.4583, -0.4185,  0.1641],
        [ 0.2271,  0.3897, -0.4541, -0.1055,  0.0229,  0.2760, -0.5044,  0.3526,
          0.0470, -0.3880,  0.3661, -0.3373,  0.5152, -0.3748,  0.3379,  0.2819,
          0.4976,  0.4345, -0.0954,  0.3253],
        [ 0.2290,  0.0803,  0.0486, -0.1841,  0.0380, -0.0146, -0.4984,  0.0810,
         -0.1812, -0.2237, -0.0205, -0.1833,  0.2845, -0.5975,  0.3312,  0.2858,
          0.4429,  0.5434, -0.3292,  0.2672]], grad_fn=<AddmmBackward0>)


After ReLU : tensor([[0.4024, 0.5842, 0.0000, 0.0000, 0.0062, 0.0806, 0.0000, 0.4268, 0.2608,
         0.0000, 0.2940, 0.0000, 0.1083, 0.0000, 0.5727, 0.1502, 0.4755, 0.4583,
         0.0000, 0.1641],
        [0.2271, 0.3897, 0.0000, 0.0000, 0.0229, 0.2760, 0.0000, 0.3526, 0.0470,
         0.0000, 0.3661, 0.0000, 0.5152, 0.0000, 0.

### nn.Softmax

신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits를 반환한다. logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0,1] 범위로 비례하여 조정(scale) 된다. <code>dim</code> 매개변수는 값의 합이 1이 되는 차원을 나타낸다.

In [10]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### 모델 매개변수

신경망 내부의 많은 계층들은 매개변수화(parameterize) 된다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어진다. <code>nn.Module</code>을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, 모델의 <code>parameters()</code> 및 <code>named_parameters()</code> 메소드로 모든 매개변수에 접근할 수 있게 된다.

이 예제에서는 각 매개변수드를 순회하며(iterate), 매개변수의 크기와 값을 출력한다.

In [11]:
print(f"Model structure : {model} \n\n")

for name, param in model.named_parameters() :
    print(f"Layer : {name} | Size : {param.size()} | Values : {param[:2]} \n")

Model structure : NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer : linear_relu_stack.0.weight | Size : torch.Size([512, 784]) | Values : tensor([[-0.0286,  0.0148,  0.0208,  ..., -0.0063,  0.0089,  0.0021],
        [ 0.0043, -0.0108, -0.0176,  ..., -0.0276, -0.0009,  0.0223]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.0.bias | Size : torch.Size([512]) | Values : tensor([ 0.0029, -0.0098], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.2.weight | Size : torch.Size([512, 512]) | Values : tensor([[ 0.0309,  0.0379, -0.0233,  ...,  0.0327, -0.0302,  0.0413],
        [-0.0290,  0.0158, -0.0391,  ...,  0.0113,  0.0130, -0.0145]],
       device='cuda:0', gra